In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- Parámetros ---
tickers = ["HOTEL.MX", "PINFRA.MX", "TLEVISACPO.MX", "GFINBURO.MX", "GCARSOA1.MX"]
pesos = np.array([0.1, 0.2, 0.15, 0.3, 0.25])
inicio = "2025-09-08"
fin = "2025-09-24"
inversion_inicial = 1_000_000 * 18.5

# --- Descargar precios ---
# yfinance puede devolver un DataFrame con niveles ("Adj Close", ticker) o con columnas simples.
# Ser robustos: descargar y preferir "Adj Close", luego "Close", y manejar Series -> DataFrame.
data = yf.download(tickers, start=inicio, end=fin, progress=False, auto_adjust=False)

if isinstance(data, pd.DataFrame) and "Adj Close" in data:
    data = data["Adj Close"].copy()
elif isinstance(data, pd.DataFrame) and "Close" in data:
    data = data["Close"].copy()
else:
    # Si yfinance devolvió una Series (un solo ticker) o ya un DataFrame de precios
    data = data.copy()

# Asegurarnos de tener un DataFrame con columnas por ticker
if isinstance(data, pd.Series):
    data = data.to_frame()

# Normalizar nombres de columnas para que coincidan con la lista 'tickers' (insensible a mayúsculas)
def normalize_col(col):
    col_str = str(col)
    for t in tickers:
        if col_str.upper() == t.upper():
            return t
        if col_str.split(".")[0].upper() == t.split(".")[0].upper():
            return t
    return col

data.columns = [normalize_col(c) for c in data.columns]

# --- Rendimientos diarios ---
rend_diarios = data.pct_change().dropna()

# --- Rendimiento acumulado por activo ---
rend_acum = (1 + rend_diarios).prod() - 1

# --- Rendimiento portafolio ---
rend_port_diario = rend_diarios @ pesos
rend_port_total = (1 + rend_port_diario).prod() - 1

# --- Riesgo (desviación estándar) ---
riesgo_port = rend_port_diario.std() * np.sqrt(len(rend_port_diario))  # aprox anualizado corto

# --- Valor final ---
valor_final = inversion_inicial * (1 + rend_port_total)

print("\n--- Resultados ---")
print("Rendimiento acumulado por activo (%):")
print(rend_acum * 100)
print("\nRendimiento total portafolio: {:.2f}%".format(rend_port_total * 100))
print("Riesgo (desviación estándar aprox.): {:.2f}%".format(riesgo_port * 100))
print("Valor final del portafolio: ${:,.2f}".format(valor_final))

# --- CAPM / Fama-French simplificado ---
# Usamos el índice IPC México como proxy del mercado (^MXX en Yahoo)
bdata = yf.download("^MXX", start=inicio, end=fin, progress=False, auto_adjust=False)

if isinstance(bdata, pd.DataFrame) and "Adj Close" in bdata:
    benchmark = bdata["Adj Close"].copy()
elif isinstance(bdata, pd.DataFrame) and "Close" in bdata:
    benchmark = bdata["Close"].copy()
elif isinstance(bdata, pd.Series):
    benchmark = bdata.copy()
else:
    # fallback: intentar extraer la primera columna si existe
    try:
        benchmark = bdata.iloc[:, 0].copy()
    except Exception:
        raise KeyError("No se pudo extraer 'Adj Close' ni 'Close' del benchmark descargado.")

# Asegurarnos de que `benchmark` sea una Serie 1-D (no un DataFrame con una sola columna)
# Si `benchmark` es un DataFrame con una sola columna, extraerla como Series.
# Si es ya una Series, dejarla intacta. En otros casos, usar squeeze() y convertir a Series.
if isinstance(benchmark, pd.DataFrame):
    if benchmark.shape[1] == 1:
        benchmark = benchmark.iloc[:, 0]
    else:
        # fallback: tomar la primera columna si hay varias
        benchmark = benchmark.iloc[:, 0]

# Asegurar que sea Series 1-D
if not isinstance(benchmark, pd.Series):
    benchmark = pd.Series(benchmark).squeeze()

# Calcular retornos y alinear índices con rend_port_diario
benchmark = benchmark.pct_change().dropna()

# Alinear rend_port_diario con las fechas del benchmark (evita problemas de desajuste)
rp_aligned = rend_port_diario.reindex(benchmark.index)

df_ff = pd.DataFrame({
    "Rp": rp_aligned,
    "Rm": benchmark
}).dropna()

X = sm.add_constant(df_ff["Rm"])
y = df_ff["Rp"]
modelo_ff = sm.OLS(y, X).fit()

print("\n--- Modelo Fama (CAPM simplificado con IPC México) ---")
print(modelo_ff.summary())



--- Resultados ---
Rendimiento acumulado por activo (%):
GCARSOA1.MX      4.408903
GFINBURO.MX      2.788692
HOTEL.MX         2.339179
PINFRA.MX        4.082304
TLEVISACPO.MX   -3.710755
dtype: float64

Rendimiento total portafolio: 1.67%
Riesgo (desviación estándar aprox.): 2.83%
Valor final del portafolio: $18,809,871.55

--- Modelo Fama (CAPM simplificado con IPC México) ---
                            OLS Regression Results                            
Dep. Variable:                     Rp   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     24.09
Date:                Mon, 29 Sep 2025   Prob (F-statistic):            0.00118
Time:                        19:20:30   Log-Likelihood:                 40.433
No. Observations:                  10   AIC:                            -76.87
Df Residuals:                       8   BIC:                     

In [1]:
df_ff

NameError: name 'df_ff' is not defined